In [1]:
import json
import asyncio

from aiohttp import ClientSession, BasicAuth
from python_opensky import OpenSky, BoundingBox
from pyensign.ensign import authenticate, publisher

# Configuration

In [2]:
OPENSKY_CREDS_PATH="opensky-creds.json"
ENSIGN_CREDS_PATH="ensign-creds.json"

def load_opensky_creds():
    with open(OPENSKY_CREDS_PATH) as f:
        creds = json.load(f)
        return BasicAuth(login=creds["username"], password=creds["password"])

OPENSKY_CREDS = load_opensky_creds()

In [3]:
BOUNDING_BOX = BoundingBox(
    min_latitude=19,
    max_latitude=65,
    min_longitude=-162,
    max_longitude=-67,
)

INTERVAL_SEC=5

# Publish Flight Updates

In [4]:
@publisher("flight-positions")
async def get_flights():
    async with ClientSession() as session:
        async with OpenSky(session=session) as opensky:
            await opensky.authenticate(OPENSKY_CREDS)
            
            response = await opensky.get_states(bounding_box=BOUNDING_BOX)
            for vector in response.states:
                yield {
                    "icao24": vector.icao24,
                    "callsign": vector.callsign,
                    "origin_country": vector.origin_country,
                    "time_position": vector.time_position,
                    "last_contact": vector.last_contact,
                    "longitude": vector.longitude,
                    "latitude": vector.latitude,
                    "geo_altitude": vector.geo_altitude,
                    "on_ground": vector.on_ground,
                    "velocity": vector.velocity,
                    "true_track": vector.true_track,
                    "vertical_rate": vector.vertical_rate,
                    "sensors": vector.sensors,
                    "barometric_altitude": vector.barometric_altitude,
                    "transponder_code": vector.transponder_code,
                    "special_purpose_indicator": vector.special_purpose_indicator,
                    "position_source": vector.position_source,
                    "category": vector.category,
                }
                
@authenticate(cred_path=ENSIGN_CREDS_PATH)
async def poll_flights():
    for i in range(10):
        async for data in get_flights():
            print(data)
        await asyncio.sleep(INTERVAL_SEC)
            
    # Wait for pending events to get published
    await asyncio.sleep(1)

In [5]:
await poll_flights()

{'icao24': 'ac96b8', 'callsign': 'AAL2267 ', 'origin_country': 'United States', 'time_position': 1697424997, 'last_contact': 1697424997, 'longitude': -95.8484, 'latitude': 36.9159, 'geo_altitude': 9029.7, 'on_ground': False, 'velocity': 189.03, 'true_track': 16.12, 'vertical_rate': 0.0, 'sensors': None, 'barometric_altitude': 8839.2, 'transponder_code': '3452', 'special_purpose_indicator': False, 'position_source': <PositionSource.ADSB: 0>, 'category': <AircraftCategory.LARGE: 4>}
{'icao24': 'aa56db', 'callsign': 'UAL2199 ', 'origin_country': 'United States', 'time_position': 1697424997, 'last_contact': 1697424997, 'longitude': -113.9584, 'latitude': 35.5558, 'geo_altitude': 12161.52, 'on_ground': False, 'velocity': 231.29, 'true_track': 248.74, 'vertical_rate': 0.0, 'sensors': None, 'barometric_altitude': 11582.4, 'transponder_code': None, 'special_purpose_indicator': False, 'position_source': <PositionSource.ADSB: 0>, 'category': <AircraftCategory.LARGE: 4>}
{'icao24': 'a2e5ec', 'cal

CancelledError: 